In [12]:
import yfinance as yf
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
def get_prices(ticker, start="2023-01-01", end="2025-01-01"):
    data = yf.download(ticker, start=start, end=end, progress=False)
    if data.empty:
        raise ValueError(f"No data returned for {ticker}")
    if "Adj Close" in data.columns:
        return data["Adj Close"]
    else:
        print(f"⚠️ 'Adj Close' not found for {ticker}, using 'Close' instead")
        return data["Close"]

# Fetch separately
ko = get_prices("KO")
pep = get_prices("PEP")

# Merge into one DataFrame
data = pd.concat([ko, pep], axis=1)
data.columns = ["KO", "PEP"]

print(data.head())

⚠️ 'Adj Close' not found for KO, using 'Close' instead
⚠️ 'Adj Close' not found for PEP, using 'Close' instead
                   KO         PEP
Date                             
2023-01-03  57.923157  164.087463
2023-01-04  57.895550  163.685028
2023-01-05  57.233051  161.974762
2023-01-06  58.337223  165.633148
2023-01-09  57.610313  164.014282


C:\Users\jmlud\AppData\Local\Temp\ipykernel_908\289051401.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, progress=False)
C:\Users\jmlud\AppData\Local\Temp\ipykernel_908\289051401.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, progress=False)
